In [1]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import random
import statsmodels.api as sm
from statsmodels import regression
%pylab inline

def linreg(x,y):
    
    #runs the regression
    x1 = sm.add_constant(x)
    model = regression.linear_model.OLS(y,x1).fit()
    
#   returns the beta, alpha and model summary
    return model.params[0], model.params[1], model.summary2()


def betas():
    
    #Gather user information
    
    benchmark = input('Which benchmark ticker would you like to use? \nSPY, Russel etc...\n')
    
    rf = float(input('What would you like to use for risk free rate?\n')) 
    #if you want to use a treasury bond, uncomment this code
#     rf = input('What would you like to use for risk free rate?\n IRX: 13 Week \n FVX: 5 Year \nTNX: 10 Year\n') 
#     rf = '^'+rf

    num_stocks = int(input('How many companies do you want to look at?\n'))
    
    stocks = [benchmark]
    
    #Gets the ticker symbols for the stocks
    while num_stocks != 0:
        s = input('please enter ticker symbol for one stock:\n')
        stocks.append(s)
        num_stocks -= 1
    start_year = input('What year do you want to start on?\nFormat: MM/DD/YYYY\n ')
    end_year = input('What year do you want to end on?\nFormat: MM/DD/YYYY\n  ')
    
    #scrape the info using yahoo finance 
    data = (web.DataReader(stocks,data_source='yahoo',start=start_year, 
                       end=end_year)['Adj Close'])
    
    #get monthly returns if you do not want to use daily returns
#     data = data.groupby(pd.TimeGrouper(freq='MS'))[stocks].mean()
        
    #create and name an excel notebook
    writer = pd.ExcelWriter('capm.xlsx', engine='xlsxwriter')
    workbook  = writer.book
    
    #subtract risk free rate from the benchmark 
    #not sure if we need to subtract risk free our not
    data[benchmark] = data[benchmark] - rf

    for stock in stocks[1:]:
        #subtract risk free rate from each stock
        #not sure if we need to subtract risk free or not
        data[stock] = data[stock] - rf
        
        #calculate the returns for the market and the stock
        returns = data[[benchmark, stock]].pct_change()
        returns = returns.dropna()
        
        #create my X and y to run a regression on
        y = returns[stock]
        X = returns[benchmark]
        
        #returns alpha, beta and the summary of regression
        a, b, model = linreg(X,y)
        
        #create a dataframe for the first part of the summary
        df = model.tables[0]
        
        #create a dataframe for the second part of the summary
        df1 = model.tables[1]
        
        #add the two dataframes together so there is only one dataframe to add to the excel spreedsheet
        table = df.append(df1)
        
        #write the dataframe to excel
        table.to_excel(writer, sheet_name='{}'.format(stock)) 
        
        #create plot of security characteristic line
        plt.scatter(X,y, alpha=0.3)
        p = np.linspace(X.min(),X.max(), 100)
        
        #b = beta and a = alpha
        y1 = b*p + a
        plt.plot(p, y1, 'r', alpha=.9)
        plt.xlabel('Market Return')
        plt.ylabel('{} return'.format(stock))
        plt.savefig('{}.png'.format(stock))
        
        #save the plot to the spreedsheet
        worksheet = writer.sheets['{}'.format(stock)]
        worksheet.insert_image('A15', '{}.png'.format(stock))
        
        #clears the plot so that each plot is unique for the designated stock
        plt.clf()
        
    #save and close the excel writer
    writer.save()
    writer.close()

#run the program
betas()

C:\Users\19785\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Populating the interactive namespace from numpy and matplotlib
Which benchmark ticker would you like to use? 
SPY, Russel etc...
spy
What would you like to use for risk free rate?
2
How many companies do you want to look at?
3
please enter ticker symbol for one stock:
tsla
please enter ticker symbol for one stock:
ibm
please enter ticker symbol for one stock:
aapl
What year do you want to start on?
Format: MM/DD/YYYY
 01/01/2010
What year do you want to end on?
Format: MM/DD/YYYY
  12/13/2019


C:\Users\19785\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<Figure size 432x288 with 0 Axes>